In [7]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

using TSSOS

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

$
X J_2 = \begin{pmatrix} d & 0 \\ 0 & -d 
   \end{pmatrix} = 
   \begin{pmatrix} \sqrt{\gamma_2} & 0 \\ 0 & -\sqrt{\gamma_2} 
   \end{pmatrix}
$

$
X J_3 = \begin{pmatrix} 0 & 0 \\ e & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 &  0 \\ \sqrt{\gamma_3} & 0 
   \end{pmatrix}
$

$
X J_4 = \begin{pmatrix} 0 & p \\ p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
X J_5 = \begin{pmatrix} 0 & -i p \\ i p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
X J_6 = \begin{pmatrix} p & 0 \\ 0 & -p
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [2]:
# Define polynomial variables
@polyvar w α β r ϕ e p

Hˢʸᵐᵇ = [ w   0
          0   0. ]

#Hˢʸᵐᵇ = [ w           α + im* β
#          α + im* β   0.        ]

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]

J₂ˢʸᵐᵇ = ϕ * [ 1    0
               0   -1. ]

J₃ˢʸᵐᵇ = [ 0   0
           e   0. ]

J₄ˢʸᵐᵇ = p * [ 0    1
               1    0. ]

J₅ˢʸᵐᵇ = p * [ 0   -im
               im   0  ]

J₆ˢʸᵐᵇ = p * [ 1    0
               0   -1. ]


function g_objective_2(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ])#, J₃ˢʸᵐᵇ, J₄ˢʸᵐᵇ, J₅ˢʸᵐᵇ, J₆ˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective_2 (generic function with 1 method)

In [3]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end


function tssos_min(p)
    
    ################################################################################################
    #
    #   Try TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    pd = p / maximum(abs.(coefficients(p)))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(pd, variables(pd), QUIET=true, solution=true, newton=true);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_tssos = previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing

    end

    best_solution = variables(p) => minimizer_tssos

end

function tssos_scaled(p::Polynomial)

    pd = p / maximum(abs.(coefficients(p)))

    ################################################################################################
    #
    #   TSSOS on polynomial with scaled variables
    #
    ################################################################################################

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    # minimize
    # minimizer_scaled_tssos = nothing

    try
        opt,sol,data = tssos_first(p_scaled, variables(p), QUIET=true, solution=true, newton=false);
        previous_sol = sol

        while ~isnothing(sol)
            previous_sol = sol
            opt,sol,data = tssos_higher!(data; QUIET=true, solution=true);
        end

        global minimizer_scaled_tssos = scale .* previous_sol
    
    catch
        println("Scaled TSSOS failed")
        global minimizer_scaled_tssos = nothing
    end

    solution = variables(p) => minimizer_scaled_tssos

    return solution
end


tssos_scaled (generic function with 1 method)

In [5]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;


../DATA/


In [6]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_2_tssos_scaled_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective_2(γᵢ)
    
    sol = tssos_scaled(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    relaxation = subs(r^2, sol)
    #de_phasing = subs(ϕ^2, sol)
    #exitation = subs(e^2, sol) 
    #de_polarisation = subs(p^2, sol)
    omega = subs(w, sol)

    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)
    J₂ˢⁱᵈ = subs(J₂ˢʸᵐᵇ, sol)
    #J₃ˢⁱᵈ = subs(J₃ˢʸᵐᵇ, sol) 

    #J₄ˢⁱᵈ = subs(J₄ˢʸᵐᵇ, sol)
    #J₅ˢⁱᵈ = subs(J₅ˢʸᵐᵇ, sol)
    #J₆ˢⁱᵈ = subs(J₆ˢʸᵐᵇ, sol)


    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group   

        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        γ_group["gamma_de_phasing"] = convert(Float64,  de_phasing)
        #γ_group["gamma_exitation"] = convert(Float64,  exitation)
        #γ_group["gamma_de_polarisation"] = convert(Float64, de_polarisation)
        
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        #γ_group["J1"] = convert.(ComplexF64, J₁ˢⁱᵈ)
        #γ_group["J2"] = convert.(ComplexF64, J₂ˢⁱᵈ)
        #γ_group["J3"] = convert.(ComplexF64, J₃ˢⁱᵈ)
        #γ_group["J4"] = convert.(ComplexF64, J₄ˢⁱᵈ)
        #γ_group["J5"] = convert.(ComplexF64, J₅ˢⁱᵈ)
        #γ_group["J6"] = convert.(ComplexF64, J₆ˢⁱᵈ)
        
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), 
                    [ DenseOperator(basis,J₁ˢⁱᵈ), DenseOperator(basis,J₂ˢⁱᵈ)]) #, DenseOperator(basis,J₃ˢⁱᵈ),
                    # DenseOperator(basis,J₄ˢⁱᵈ), DenseOperator(basis,J₅ˢⁱᵈ), DenseOperator(basis,J₆ˢⁱᵈ)])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...


optimum = 2.555623786484264e-6


Global optimality certified with relative optimality gap 0.002122%!
optimum = 2.52927897328515e-6
Global optimality certified with relative optimality gap 0.002125%!
No higher TS step of the TSSOS hierarchy!


State_D1 0.9453782543395463

...

0.9995608482046809
State_D6 0.9995608482046138
State_D7 0.9976894884850938
State_D8 0.9990714934696997
State_D9 0.9992552340967324
State_D10 0.9994436677172946
State_D11 

0.9991674718156732
State_D12 0.9994878550659978
State_D13 0.9990345414327348
State_D14 0.9994842331067939
State_D15 0.9997122953868114
State_D16 0.9997122953867976
State_D17 

0.9976894884850938
State_D18 0.9990714934696636
State_D19 0.9994436677173253
State_D20 0.999255234096655

γ =  251.33


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.9999256250994023
Global optimality certified with relative optimality gap 0.001605%!
optimum = 0.9999256319699027
Global optimality certified with relative optimality gap 0.001604%!
No higher TS step of the TSSOS hierarchy!
State_D1 0.9965356821359648
State_D2 0.9951092850000244
State_D3 0.9957899341877683
State_D4 0.9944258961142087
State_D5 0.993462939643099
State_D6 0.993462939643099
State_D7 0.9996798923905036
State_D8 0.998272716381304
State_D9 0.9947920896441605
State_D10 0.993632846152415
State_D11 0.9957899341877547


State_D12 0.9944258961142087
State_D13 0.9965356821359648
State_D14 0.9951092850000874
State_D15 0.9930190594878627
State_D16 0.9930190594878165
State_D17 0.9996798923905147
State_D18 0.9982727163813007
State_D19 0.993632846152415
State_D20 0.994792089644052

POP_LME_2_tssos_scaled_trn4_tst20_2024-Jan-16_at_13-49.h5
